# Final Project: ASL Recognition
### Professor: Weizhe Li
### Student: Levan Sulimanov

# Training Pipeline - LSTM Module

# Main Imports:

In [1]:
# main imports
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)
import time

# Train LSTM Model:

In [2]:
# pip install ipywidgets widgetsnbextension pandas-profiling

In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
import torch
print(torch.__version__)
import pytorch_lightning
print(pytorch_lightning.__version__)

1.12.1+cu116
1.8.3.post1


# Training Pipeline - LSTM Module

In [5]:
# main imports
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)
import time

# Visualization Utilities:

In [6]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.tickers import SingleIntervalTicker
output_notebook()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

p1 = figure(title="Loss", x_axis_label="Epoch", height=300, width=360)
p2 = figure(title="Accuracy", x_axis_label="Epoch", height=300, width=360)

source1 = ColumnDataSource(data={'epochs': [], 'trainlosses': [], 'testlosses': [] })
source2 = ColumnDataSource(data={'epochs': [], 'train_accuracies': [], 'test_accuracies': []})

#r = p1.multi_line(ys=['trainlosses', 'testlosses'], xs='epochs', color=colors, alpha=0.8, legend_label=['Training','Test'], source=source)
r1 = p1.line(x='epochs', y='trainlosses', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source1)
r2 = p1.line(x='epochs', y='testlosses', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source1)

r3 = p2.line(x='epochs', y='train_accuracies', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source2)
r4 = p2.line(x='epochs', y='test_accuracies', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source2)

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p2.legend.location = "bottom_right"
p2.legend.click_policy="hide"

Loading BokehJS ...

# Data Loader and Model Initialization:

In [7]:
###################################### CREDITS ######################################
# https://learnopencv.com/human-action-recognition-using-detectron2-and-lstm/
#####################################################################################

import numpy as np
import torch.optim as optim
import torchmetrics
import pytorch_lightning as pl

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
from numpy import genfromtxt

class PoseDataset(Dataset):
    def __init__(self, data_arr):
        self.data_arr = data_arr

    def __len__(self):
        return len(self.data_arr)

    def __getitem__(self, idx):
        X_path = self.data_arr[idx][0]
        y = self.data_arr[idx][1]
        
        X = torch.from_numpy(genfromtxt(X_path, delimiter=',')).float()
        
        # print("X:", X.shape)
        # print("y:", y)
        
        return X, y


class PoseDataModule(pl.LightningDataModule):
    def __init__(self, data_root, batch_size=32):
        super().__init__()

        self.data_root = data_root
        self.batch_size = batch_size


    def load_X_and_Y(self, data_path, train_mode="train"):
        
        data_x_y_arr = []
        
        read_from_dir = os.path.join(data_path, train_mode)
        
        # go over each class folder, get subsamples per class, and return tuples [sample_path, class_number]
        for class_folder in os.listdir(read_from_dir):
            class_abs_path = os.path.join(read_from_dir, class_folder)
            for sample in os.listdir(class_abs_path):
                sample_path = os.path.join(class_abs_path, sample)
                class_num = class_to_label_num[class_folder]
                data_x_y_arr.append([sample_path, class_num])

        # print(f"Collected full list for {train_mode}:\n{data_x_y_arr}")
        return data_x_y_arr
            

    def setup(self, stage=None):
        train_data_arr = self.load_X_and_Y(self.data_root, train_mode="train")
        eval_data_arr = self.load_X_and_Y(self.data_root, train_mode="val")
        # test_data_arr = self.load_X_and_Y(self.data_root, train_mode="test")
        self.train_dataset = PoseDataset(train_data_arr)
        self.val_dataset = PoseDataset(eval_data_arr)
        # self.test_dataset = PoseDataset(test_data_arr)

    def train_dataloader(self):
        # train loader
        train_loader = torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=0
        )
        return train_loader

    def val_dataloader(self):
        # validation loader
        val_loader = torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=0
        )
        return val_loader
    '''
    def test_dataloader(self):
        # validation loader
        test_loader = torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=8
        )
        return test_loader
    '''

# We have 6 output action classes.
TOT_ACTION_CLASSES = 9 #6

#lstm classifier definition
class ActionClassificationLSTM(pl.LightningModule):
    # initialise method
    def __init__(self, input_features, hidden_dim, learning_rate=0.001):
        super().__init__()
        # save hyperparameters
        self.save_hyperparameters()
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(input_features, hidden_dim, batch_first=True)
        # The linear layer that maps from hidden state space to classes
        self.linear = nn.Linear(hidden_dim, TOT_ACTION_CLASSES)

    def forward(self, x):
        # invoke lstm layer
        lstm_out, (ht, ct) = self.lstm(x)
        # invoke linear layer
        return self.linear(ht[-1])

    def training_step(self, batch, batch_idx):
        # get data and labels from batch
        x, y = batch
        # reduce dimension
        y = torch.squeeze(y)
        # convert to long
        y = y.long()
        # get prediction
        y_pred = self(x)
        # calculate loss
        loss = F.cross_entropy(y_pred, y)
        # get probability score using softmax
        prob = F.softmax(y_pred, dim=1)
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1]
        # calculate accuracy
        acc = torchmetrics.functional.accuracy(pred, y, task="multiclass", num_classes=TOT_ACTION_CLASSES)
        dic = {
            'batch_train_loss': loss,
            'batch_train_acc': acc
        }
        # log the metrics for pytorch lightning progress bar or any other operations
        self.log('batch_train_loss', loss, prog_bar=True)
        self.log('batch_train_acc', acc, prog_bar=True)
        #return loss and dict
        return {'loss': loss, 'result': dic}

    def training_epoch_end(self, training_step_outputs):
        # calculate average training loss end of the epoch
        avg_train_loss = torch.tensor([x['result']['batch_train_loss'] for x in training_step_outputs]).mean()
        # calculate average training accuracy end of the epoch
        avg_train_acc = torch.tensor([x['result']['batch_train_acc'] for x in training_step_outputs]).mean()
        # log the metrics for pytorch lightning progress bar and any further processing
        self.log('train_loss', avg_train_loss, prog_bar=True)
        self.log('train_acc', avg_train_acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        # get data and labels from batch
        x, y = batch
        # reduce dimension
        y = torch.squeeze(y)
        # convert to long
        y = y.long()
        # get prediction
        y_pred = self(x)
        # calculate loss
        loss = F.cross_entropy(y_pred, y)
        # get probability score using softmax
        prob = F.softmax(y_pred, dim=1)
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1]
        # calculate accuracy
        acc = torchmetrics.functional.accuracy(pred, y, task="multiclass", num_classes=TOT_ACTION_CLASSES)
        dic = {
            'batch_val_loss': loss,
            'batch_val_acc': acc
        }
        # log the metrics for pytorch lightning progress bar and any further processing
        self.log('batch_val_loss', loss, prog_bar=True)
        self.log('batch_val_acc', acc, prog_bar=True)
        #return dict
        return dic

    def validation_epoch_end(self, validation_step_outputs):
        # calculate average validation loss end of the epoch
        avg_val_loss = torch.tensor([x['batch_val_loss']
                                     for x in validation_step_outputs]).mean()
        # calculate average validation accuracy end of the epoch
        avg_val_acc = torch.tensor([x['batch_val_acc']
                                    for x in validation_step_outputs]).mean()
        # log the metrics for pytorch lightning progress bar and any further processing
        self.log('val_loss', avg_val_loss, prog_bar=True)
        self.log('val_acc', avg_val_acc, prog_bar=True)

    def configure_optimizers(self):
        # adam optimiser
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        # learning rate reducer scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, min_lr=1e-15, verbose=True)
        # scheduler reduces learning rate based on the value of val_loss metric
        return {"optimizer": optimizer,
                "lr_scheduler": {"scheduler": scheduler, "interval": "epoch", "frequency": 1, "monitor": "val_loss"}}

# Defining Class names and labels:

In [8]:
class_names_lst = ["hello", "my", "world", "me", "every", "moment", "is", "new", "beginning"]
class_to_label_num = {"hello": 0, "my": 1, "world": 2, "me": 3, "every": 4, "moment": 5, "is": 6, "new": 7, "beginning": 8}

# read in available data samples:
main_dir = os.getcwd()
data_dir = os.path.join(main_dir, "keypoints_data")

In [9]:
DATASET_PATH = data_dir
print(DATASET_PATH)

C:\Users\lrspr\Desktop\Masters_Program\690_Deep_Learning\Projects\Project_3_due_12_05_22\keypoints_data


# Training pipeline *adapted* from:
##### https://learnopencv.com/human-action-recognition-using-detectron2-and-lstm/

In [10]:
from argparse import ArgumentParser

def configuration_parser(parent_parser):
    parser = ArgumentParser(parents=[parent_parser], add_help=False)
    parser.add_argument('--batch_size', type=int, default=512)
    parser.add_argument('--epochs', type=int, default=500)
    parser.add_argument('--data_root', type=str, default=DATASET_PATH)
    parser.add_argument('--learning_rate', type=float, default=0.0001)
    parser.add_argument('--num_class', type=int, default=TOT_ACTION_CLASSES)
    return parser

In [11]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

#from src.lstm import ActionClassificationLSTM, PoseDataModule
# ActionClassificationLSTM, PoseDataModule => included here

In [12]:
def do_training_validation():
    pl.seed_everything(21)    
    parser = ArgumentParser()
    parser = pl.Trainer.add_argparse_args(parser)
    parser = configuration_parser(parser)
    # args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    # init model
    print("args.data_root:", args.data_root)
    model = ActionClassificationLSTM(92, 124, learning_rate=args.learning_rate)
    data_module = PoseDataModule(data_root=args.data_root,
                                 batch_size=args.batch_size)    
    #save only the top 1 model based on val_loss
    checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor='val_loss')
    lr_monitor = LearningRateMonitor(logging_interval='step')  
    #trainer
    trainer = pl.Trainer.from_argparse_args(args,
        # fast_dev_run=True,
        max_epochs=args.epochs, 
        deterministic=True, 
        gpus=1, 
        # progress_bar_refresh_rate=1, 
        callbacks=[EarlyStopping(monitor='train_loss', patience=15), checkpoint_callback, lr_monitor])    
    trainer.fit(model, data_module)  
    print("Trainer Logs are:\n")
    print(trainer.logged_metrics)
    return model

In [2]:
# To reload tensorBoard
%load_ext tensorboard

# logs folder path
# %tensorboard --logdir=lightning_logs
%tensorboard --logdir=lightning_logs/ --host localhost --port 8090

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
do_training_validation()

Global seed set to 21
C:\Users\lrspr\anaconda3\envs\asl_rec\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True


args.data_root: C:\Users\lrspr\Desktop\Masters_Program\690_Deep_Learning\Projects\Project_3_due_12_05_22\keypoints_data


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | lstm   | LSTM   | 108 K 
1 | linear | Linear | 1.1 K 
----------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.437     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\lrspr\anaconda3\envs\asl_rec\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\lrspr\anaconda3\envs\asl_rec\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\lrspr\anaconda3\envs\asl_rec\lib\site-packages\pytorch_lightning\trainer\trainer.py:1558: PossibleUserWarning: The number of training batches (5) is smaller than the logging inter

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 00343: reducing learning rate of group 0 to 9.7656e-08.
Trainer Logs are:

{'batch_train_loss': tensor(0.1319), 'batch_train_acc': tensor(0.9766), 'batch_val_loss': tensor(0.3376), 'batch_val_acc': tensor(0.8990), 'val_loss': tensor(0.3376), 'val_acc': tensor(0.8990), 'train_loss': tensor(0.1309), 'train_acc': tensor(0.9762)}


ActionClassificationLSTM(
  (lstm): LSTM(92, 124, batch_first=True)
  (linear): Linear(in_features=124, out_features=9, bias=True)
)

In [19]:
import os
def get_latest_run_version_ckpt_epoch_no(lightning_logs_dir='lightning_logs', run_version=None):
    if run_version is None:
        run_version = 0
        for dir_name in os.listdir(lightning_logs_dir):
            if 'version' in dir_name:
                if int(dir_name.split('_')[1]) > run_version:
                    run_version = int(dir_name.split('_')[1])                
    checkpoints_dir = os.path.join(lightning_logs_dir, 'version_{}'.format(run_version), 'checkpoints')    
    files = os.listdir(checkpoints_dir)
    ckpt_filename = None
    for file in files:
        print(file)
        if file.endswith('.ckpt'):
            ckpt_filename = file        
    if ckpt_filename is not None:
        ckpt_path = os.path.join(checkpoints_dir, ckpt_filename)
    else:
        print('CKPT file is not present')    
    return ckpt_path

In [20]:
ckpt_path = get_latest_run_version_ckpt_epoch_no()
print('The latest model path: {}'.format(ckpt_path))

epoch=243-step=1220.ckpt
The latest model path: lightning_logs\version_1\checkpoints\epoch=243-step=1220.ckpt
